In [2]:
%load_ext autoreload
%autoreload 2
from rxnutils import *
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import os
import ast
from rxnmapper import RXNMapper
from chemUtils.synthesis import reaction
import json
from IPython.display import display
from collections import OrderedDict
from prettytable import PrettyTable

In [63]:
with open('/Users/kate_fieseler/PycharmProjects/retrievesynthesizable/D68EV3CPROA/RXN_SMARTS_CONSTANTS.json', 'r') as f:
    reaction_smarts = json.load(f)
# Pretty print the JSON content
pretty_json = json.dumps(reaction_smarts, indent=4)
print(pretty_json)

{
    "Amidation": "[#6:1](=[#8:2])-[#8;H1].[#7&X3;!$(NC=*);!$(NS):3]>>[#6:1](=[#8:2])-[#7X3:3]",
    "Amide_schotten-baumann": "[#7;H2,H1:3].[#6:1](=[#8:2])-[#17]>>[#6:1](=[#8:2])-[#7:3]",
    "Reductive_amination": "[#6:2](=[#8])(-[#6:1]).[#7;H2,H1:3]>>[#6:2](-[#6:1])-[#7:3]",
    "N-nucleophilic_aromatic_substitution": "[#6:3]-[#7;H3,H2,H1:2].[c:1]-[F,Cl,Br,I]>>[#6:3]-[#7:2]-[c:1]",
    "Sp2-sp2_Suzuki_coupling": "[#6X3:1]-[F,Cl,Br,I].[#6X3:2]-[B](-[O])(-[O])>>[#6X3:1]-[#6X3:2]",
    "Formation_of_urea_from_two_amines": "[N&X3;H2,H1;!$(NC=O);!$(NS):3].[N&X3;H2,H1;!$(NC=O);!$(NS):4]>>[#7&X3:3]-[#6](=[#8])-[#7&X3:4]",
    "Sulfonamide_Schotten-Baumann_with_amine_(intermolecular)": "[#16X4:1](=[OX1:2])(=[OX1:3])([#6,#7:4])[#17,#9].[#7&X3;H2,H1;!$(NC=O);!$(NS):5]>>[#16X4:1](=[OX1:2])(=[OX1:3])([#6,#7:4])-[#7:5]",
    "Buchwald-Hartwig_amination": "[ar:1]-[Cl,Br,I].[#7&X3;H2,H1;!$(NC=*);!$(NS);!$(NN):7]>>[ar:1]-[N:7]",
    "Nucleophilic_substitution_with_thiol": "[Cl,Br]-[C;H2,H1;+0:1].[

In [64]:
reaction_smarts['Sp2-sp2_Suzuki_coupling']

'[#6X3:1]-[F,Cl,Br,I].[#6X3:2]-[B](-[O])(-[O])>>[#6X3:1]-[#6X3:2]'

In [75]:
# Need to check that all reactions in df1 with the name in constants works for the smarts
def check_smarts(df, name_to_check):
    smarts_dict = {}
    for i in range(len(df)):
        row = df.iloc[i]
        for j in range(row.num_steps): # perform for each step in route
            name = ast.literal_eval(row.rxn_order_first_to_last)[j]
            name = name.replace(' ', '_')
            reactants = ast.literal_eval(row.reactants)[j]
            if row.num_steps == 1:
                products = [row.smiles]
            else:
                if j < 1:
                    products = ast.literal_eval(row.reactants)[j-1]
                else:
                    products = [row.smiles]
            if name == name_to_check:
                smarts_name = name.replace(' ', '_')
                mapping = reaction_smarts[smarts_name]
                print('ROW:', i)
                print(reactants)
                if Chem.MolFromSmiles(reactants[0]).GetNumAtoms() == 1:
                    if len(reactants) == 2:
                        if Chem.MolFromSmiles(reactants[1]).GetNumAtoms() == 1:
                            print('ONE ATOM REACTANT')
                            continue
                    else:
                        print('ONE ATOM REACTANT')
                        continue
                outcome = reaction.check_rxn_template_works(mapping, reactants, products, verbose=True)
                print(outcome)
                if outcome[0] != 1:
                    print('FAILED')
                    print(row)
                    print('REACTANT(S) BELOW:')
                    display(Chem.MolFromSmiles(reactants[0]))
                    if len(reactants) == 2:
                        display(Chem.MolFromSmiles(reactants[1]))
                    if outcome[0] == 0:
                        print('THE PREDICTED PRODUCT(S) IS/ARE WRONG')
                        for i in outcome[1]:
                            display(Chem.MolFromSmiles(i))
                        print('PROPER PRODUCT IS ONE OF THE BELOW')
                        for i in products:
                            display(Chem.MolFromSmiles(i))
                    else:
                        print('THE PREDICTED PRODUCT(S) IS/ARE RIGHT')
                        for i in outcome[1]:
                            display(Chem.MolFromSmiles(i))
                        print(outcome[0])
                    break

# CHECKING ONE STEP

In [76]:
name_to_check = 'N-nucleophilic_aromatic_substitution'
os.chdir('/Users/kate_fieseler/PycharmProjects/retrievesynthesizable/D68EV3CPROA/routes_data')
df = pd.read_csv('D68EV3CPROA_enumeration_oct10_first_route_1_step_OUTPUT.csv', index_col=0)
#df = pd.read_csv('D68EV3CPROA_enumeration_oct10_first_route_1_step.csv')
#reaction_smarts[name_to_check] = "[Cl,Br,I]-[#6:1].[#6;+0:2]-[OH;D1;+0:3]>>[#6X4:1]-[O;H0;D2;+0:3]-[#6X4+0:2]"
# with open('/Users/kate_fieseler/PycharmProjects/retrievesynthesizable/D68EV3CPROA/RXN_SMARTS_CONSTANTS.json', 'w') as f:
#     json.dump(reaction_smarts, f)
    
check_smarts(df, name_to_check)

ROW: 20
('O=C(NCc1ccc2c(c1)OCO2)C1CCCNC1', 'Clc1nnc2ccccn12')
The reaction SMARTS is applicable and produces the expected product with Tanimoto similarity of 1.0.
outcome of first combo ['O=C(C1CCCNC1)N(Cc1ccc2c(c1)OCO2)c1nnc2ccccn12', 'O=C(C1CCCNC1)N(Cc1ccc2c(c1)OCO2)c1nnc2ccccn12', 'O=C(NCc1ccc2c(c1)OCO2)C1CCCN(c2nnc3ccccn23)C1', 'O=C(NCc1ccc2c(c1)OCO2)C1CCCN(c2nnc3ccccn23)C1']
The reaction SMARTS is not applicable to the provided reactants.
outcome of second combo []
(1, ['O=C(C1CCCNC1)N(Cc1ccc2c(c1)OCO2)c1nnc2ccccn12', 'O=C(C1CCCNC1)N(Cc1ccc2c(c1)OCO2)c1nnc2ccccn12', 'O=C(NCc1ccc2c(c1)OCO2)C1CCCN(c2nnc3ccccn23)C1', 'O=C(NCc1ccc2c(c1)OCO2)C1CCCN(c2nnc3ccccn23)C1'])
ROW: 32
('Cc1nnc2c(Cl)nccn12', 'CNCCc1ccc2c(c1)OCO2')
The reaction SMARTS is not applicable to the provided reactants.
outcome of first combo []
The reaction SMARTS is applicable and produces the expected product with Tanimoto similarity of 1.0.
outcome of second combo ['Cc1nnc2c(N(C)CCc3ccc4c(c3)OCO4)nccn12', 'Cc1nnc2c(N

In [69]:
reaction_smarts

{'Amidation': '[#6:1](=[#8:2])-[#8;H1].[#7&X3;!$(NC=*);!$(NS):3]>>[#6:1](=[#8:2])-[#7X3:3]',
 'Amide_schotten-baumann': '[#7;H2,H1:3].[#6:1](=[#8:2])-[#17]>>[#6:1](=[#8:2])-[#7:3]',
 'Reductive_amination': '[#6:2](=[#8])(-[#6:1]).[#7;H2,H1:3]>>[#6:2](-[#6:1])-[#7:3]',
 'N-nucleophilic_aromatic_substitution': '[#6:3]-[#7;H3,H2,H1:2].[c:1]-[F,Cl,Br,I]>>[#6:3]-[#7:2]-[c:1]',
 'Sp2-sp2_Suzuki_coupling': '[#6X3:1]-[F,Cl,Br,I].[#6X3:2]-[B](-[O])(-[O])>>[#6X3:1]-[#6X3:2]',
 'Formation_of_urea_from_two_amines': '[N&X3;H2,H1;!$(NC=O);!$(NS):3].[N&X3;H2,H1;!$(NC=O);!$(NS):4]>>[#7&X3:3]-[#6](=[#8])-[#7&X3:4]',
 'Sulfonamide_Schotten-Baumann_with_amine_(intermolecular)': '[#16X4:1](=[OX1:2])(=[OX1:3])([#6,#7:4])[#17,#9].[#7&X3;H2,H1;!$(NC=O);!$(NS):5]>>[#16X4:1](=[OX1:2])(=[OX1:3])([#6,#7:4])-[#7:5]',
 'Buchwald-Hartwig_amination': '[ar:1]-[Cl,Br,I].[#7&X3;H2,H1;!$(NC=*);!$(NS);!$(NN):7]>>[ar:1]-[N:7]',
 'Nucleophilic_substitution_with_thiol': '[Cl,Br]-[C;H2,H1;+0:1].[SH;+0:2]>>[S&X2;H0;D2;+0:2]-[

In [7]:
# Look at already created smarts patterns
with open('/Users/kate_fieseler/PycharmProjects/retrievesynthesizable/D68EV3CPROA/routes_data/all_single_step_max_smarts.json', 'r') as f:
    smarts = json.load(f)
smarts

{'Formation of urea from two amines': [['[C:1]-[NH2;D1;+0:2].[NH2;D1;+0:3]-[C:4]-[c:5]1:[c:6]:[c;H0;D3;+0:7]2:[c;H0;D3;+0:8](:[cH;D2;+0:9]:[c:10]:1)-[O;H0;D2;+0:11]-[CH2;D2;+0:12]-[#8:13]-2>>[C:1]-[NH;D2;+0:2]-[C;H0;D3;+0:8](=[O;H0;D1;+0:11])-[NH;D2;+0:3]-[C:4]-[c:5]1:[c:6]:[c;H0;D3;+0:7]2:[c:14](:[cH;D2;+0:9]:[c:10]:1)-[#8:15]-[CH2;D2;+0:12]-[#8:13]-2',
   2],
  ['[C:1]-[NH2;D1;+0:2].[NH2;D1;+0:3]-[C:4]-[c:5]1:[c:6]:[c:7]:[c;H0;D3;+0:8]2:[c;H0;D3;+0:9](:[cH;D2;+0:10]:1)-[O;H0;D2;+0:11]-[CH2;D2;+0:12]-[#8:13]-2>>[C:1]-[NH;D2;+0:2]-[C;H0;D3;+0:9](=[O;H0;D1;+0:11])-[NH;D2;+0:3]-[C:4]-[c:5]1:[c:6]:[c:7]:[c;H0;D3;+0:8]2:[c:14](:[cH;D2;+0:10]:1)-[#8:15]-[CH2;D2;+0:12]-[#8:13]-2',
   2],
  ['[C:1]-[NH2;D1;+0:2].[NH2;D1;+0:3]-[C:4]-[c:5]1:[c:6]:[c;H0;D3;+0:7]2:[c;H0;D3;+0:8](:[cH;D2;+0:9]:[c:10]:1)-[O;H0;D2;+0:11]-[CH2;D2;+0:12]-[#8:13]-2>>[C:1]-[NH;D2;+0:2]-[C;H0;D3;+0:8](=[O;H0;D1;+0:11])-[NH;D2;+0:3]-[C:4]-[c:5]1:[c:6]:[c;H0;D3;+0:7]2:[c:14](:[cH;D2;+0:9]:[c:10]:1)-[#8:15]-[CH2;D2;+0:12]-[

In [34]:
smarts['Alkene hydrogenation (unconjugated)']

KeyError: 'Alkene hydrogenation (unconjugated)'

In [27]:
test = AllChem.ReactionFromSmarts('[Cl,O=S]>>[Cl,O=S(=O)(C(F)(F)F)]')
print(test)

ValueError: ChemicalReactionParserException: Problems constructing reactant from SMARTS: [Cl,O=S]

In [6]:
os.chdir('/Users/kate_fieseler/PycharmProjects/retrievesynthesizable/D68EV3CPROA/routes_data')
df = pd.read_csv('D68EV3CPROA_enumeration_oct10_first_route_1_step_OUTPUT.csv', index_col=0)
df = df[df.rxn_order_first_to_last == "['Reductive amination']"]
df

,smiles,num_steps,rxn_order_first_to_last,reactants,BuildingBlocks,catalogName_for_BuildingBlock,dir_name,Done_Time,Analogs
9,Cc1nnc2c(NCc3ccc4c(c3)OCO4)nccn12,1,['Reductive amination'],"[('O=Cc1ccc2c(c1)OCO2', 'Cc1nnc2c(N)nccn12')]","['Cc1nnc2c(NCc3ccc4c(c3)OCO4)nccn12', 'Cc1nnc2...","['enamine_real', 'enamine_bb', 'enamine_made',...","['ENAMINE_Z1230717304_x0147_0A_x1140_0A', 'ENA...",NaN,NaN
40,O[C@@H](CNCc1nnc2ccccn12)c1ccc2c(c1)OCO2,1,['Reductive amination'],"[('O=Cc1nnc2ccccn12', 'NCC(O)c1ccc2c(c1)OCO2')]","['O=Cc1nnc2ccccn12', 'NCC(O)c1ccc2c(c1)OCO2', ...","['mcule', 'molport', 'enamine_made', 'enamine_...",['MOLPORT_009-076-414_x1140_0A_x1594_0A'],NaN,NaN
66,Cc1cc(C(=O)N[C@H](C)[C@H](C)NCc2ccccn2)no1,1,['Reductive amination'],"[('O=Cc1ccccn1', 'Cc1cc(C(=O)NC(C)C(C)N)no1')]","['Cc1cc(C(=O)NC(C)C(C)N)no1', None, 'O=Cc1cccc...","['enamine_made', 'enamine_real', 'generic', 'e...",['ENAMINE_PV-000980621569_x1052_1A_x1498_0A'],10/17/23 7:20,0.0
144,CCOC(=O)C[C@@H](c1cc(C)on1)N1CCCCC1,1,['Reductive amination'],"[('CCOC(=O)CC(=O)c1cc(C)on1', 'C1CCNCC1')]","['C1CCNCC1', 'CCOC(=O)CC(=O)c1cc(C)on1', 'CCOC...","['generic', 'mcule', 'mcule', 'enamine_made']",['CHEMSPACE-BB_CSMB00133658290_x1052_1A_x1083_...,10/17/23 10:00,0.0
154,C[C@@H](F)CCNC1CCN(C(=O)CCN2CCCCC2)CC1,1,['Reductive amination'],"[('O=C1CCN(C(=O)CCN2CCCCC2)CC1', 'CC(F)CCN')]","['CC(F)CCN', None, 'O=C1CCN(C(=O)CCN2CCCCC2)CC1']","['molport', 'mcule', 'enamine_bb', 'enamine_ma...",['ENAMINE_PV-000745136799_x1083_0A_x1071_0A'],2023-10-20 07:49:20,0.0
167,c1cncc([C@@H]2CCC[C@@H](NCc3ccc4c(c3)OCO4)C2)c1,1,['Reductive amination'],"[('O=C1CCCC(c2cccnc2)C1', 'NCc1ccc2c(c1)OCO2')]","['c1cncc(C2CCCC(NCc3ccc4c(c3)OCO4)C2)c1', 'O=C...","['enamine_real', 'enamine_bb', 'mcule', 'enami...",['ENAMINE_Z2001417850_x1498_0A_x1140_0A'],2023-10-20 08:10:25,0.0
202,COc1cccc(CN[C@H](Cc2ccc3c(c2)OCO3)C(=O)O)n1,1,['Reductive amination'],"[('NC(Cc1ccc2c(c1)OCO2)C(=O)O', 'COc1cccc(C=O)...","['NC(Cc1ccc2c(c1)OCO2)C(=O)O', 'COc1cccc(C=O)n...","['mcule', 'enamine_made', 'enamine_bb', 'mcule...",['ENAMINE_Z1860323261_x1498_0A_x1140_0A'],10/17/23 10:24,0.0
254,Cc1cc(C(=O)NCC[C@@H](C)NCC(=O)N2CCCCC2)no1,1,['Reductive amination'],"[('NCC(=O)N1CCCCC1', 'CC(=O)CCNC(=O)c1cc(C)on1')]","['NCC(=O)N1CCCCC1', 'CC(=O)CCNC(=O)c1cc(C)on1'...","['enamine_real', 'enamine_bb', 'generic', 'mol...","['ENAMINE_PV-000791373338,ENAMINE_PV-000968064...",10/17/23 11:35,0.0
260,FC(F)(F)C(F)(F)[C@@H]1CCCN(CCCN2CCCCC2)C1,1,['Reductive amination'],"[('O=CCCN1CCCCC1', 'FC(F)(F)C(F)(F)C1CCCNC1')]","['FC(F)(F)C(F)(F)C1CCCN(CCCN2CCCCC2)C1', 'O=CC...","['enamine_real', 'mcule', 'enamine_made', 'mol...",['ENAMINE_Z3882913443_x1083_0A_x1071_0A'],10/17/23 11:40,0.0
293,CC(C)(C)c1cncc(CN[C@@H](CN2CCCCC2)C(=O)O)c1,1,['Reductive amination'],"[('NC(CN1CCCCC1)C(=O)O', 'CC(C)(C)c1cncc(C=O)c...","[None, 'NC(CN1CCCCC1)C(=O)O', 'CC(C)(C)c1cncc(...","['enamine_made', 'molport', 'enamine_bb', 'mcu...",['ENAMINE_Z3255855012_x1498_0A_x1083_0A'],10/17/23 12:18,0.0
